In [ ]:
from easydict import EasyDict as edict
import numpy as np
import pandas as pd
import utils.waypoint_extract_utils as weu
import utils.data_preprocess_utils as dpu
import utils.visualize_utils as vu

In [ ]:
cfg = edict(dpu.get_config('./cfg/waypoint_extraction_cfg.yaml'))
data_dir = cfg.data_dir
files = dpu.get_all_files(data_dir)
def get_subset(files_path, number):
    idx = np.random.choice(len(files_path), number, replace=False)
    # return a concated dataframe
    return pd.concat([pd.read_feather(files_path[i]) for i in idx], ignore_index=True)
df = get_subset(files, 6)
df = df.sort_values(by=['MMSI', 'time']).reset_index(drop=True)
simplified_trajs = df.groupby('MMSI').apply(weu.simplify_traj).reset_index(drop=True)
# maneuver_points = weu.dicover_maneuver_points(df, cfg)
# total_feature_points = pd.concat([simplified_trajs, maneuver_points], ignore_index=True).reset_index(drop=True)
total_feature_points = simplified_trajs
print('total_feature_points', total_feature_points.shape)
total_feature_points = total_feature_points.drop_duplicates(subset=['MMSI', 'time']).reset_index(drop=True)
print('total_feature_points', total_feature_points.shape)
feature_points_labels = weu.clustering_hdbscan(total_feature_points, cfg)
total_feature_points['cluster_label'] = feature_points_labels
total_feature_points.to_feather(cfg.out_dir)